In [ ]:
!pip install timm

In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import timm

C:\Users\16213\PycharmProjects\learn_cuda\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# final

In [3]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DTYPE = torch.float32

In [4]:
BATCH_SIZE = 75

epochs = 40

lr = 0.001
momentum = 0.9

In [5]:
for i in range(14):
    print("ne["+str(i*14)+":"+str(i*14+16)+"]",end=",")

print()
for i in range(14):
    print("ne1[:,"+str(i*14)+":"+str(i*14+16)+"]",end=",")

ne[0:16],ne[14:30],ne[28:44],ne[42:58],ne[56:72],ne[70:86],ne[84:100],ne[98:114],ne[112:128],ne[126:142],ne[140:156],ne[154:170],ne[168:184],ne[182:198],
ne1[:,0:16],ne1[:,14:30],ne1[:,28:44],ne1[:,42:58],ne1[:,56:72],ne1[:,70:86],ne1[:,84:100],ne1[:,98:114],ne1[:,112:128],ne1[:,126:142],ne1[:,140:156],ne1[:,154:170],ne1[:,168:184],ne1[:,182:198],

In [6]:
class MyTransform(object):
    def __init__(self):
        pass

    def __call__(self, img):
        ne = np.array(torchvision.transforms.ToPILImage()(img))
        ne1 = np.vstack((ne[0:16],ne[14:30],ne[28:44],ne[42:58],ne[56:72],ne[70:86],ne[84:100],ne[98:114],ne[112:128],ne[126:142],ne[140:156],ne[154:170],ne[168:184],ne[182:198]))
        ne2 = np.hstack((ne1[:,0:16],ne1[:,14:30],ne1[:,28:44],ne1[:,42:58],ne1[:,56:72],ne1[:,70:86],ne1[:,84:100],ne1[:,98:114],ne1[:,112:128],ne1[:,126:142],ne1[:,140:156],ne1[:,154:170],ne1[:,168:184],ne1[:,182:198]))
        return Image.fromarray(ne2)

In [7]:
#torchvision.transforms.Resize(32),
transform1 = torchvision.transforms.Compose([   torchvision.transforms.RandomHorizontalFlip(),
                                                torchvision.transforms.ToTensor(),
                                                torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                                                torchvision.transforms.Resize(198),
                                                MyTransform(),
                                                torchvision.transforms.ToTensor(),
                                                torchvision.transforms.ConvertImageDtype(DTYPE)
                                             ])

transform2 = torchvision.transforms.Compose([
                                                torchvision.transforms.ToTensor(),
                                                torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                                                torchvision.transforms.Resize(198),
                                                MyTransform(),
                                                torchvision.transforms.ToTensor(),
                                                torchvision.transforms.ConvertImageDtype(DTYPE)
                                             ])

In [8]:
# number of workers != 0 will lead [32]broken pipeline
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform1)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform2)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=0)

Files already downloaded and verified
Files already downloaded and verified


In [10]:
#net = torch.load("checkpoint/vit_base_patch16_224.pth")
net = timm.create_model("vit_base_patch16_224", pretrained=True)
net.head = torch.nn.Linear(net.head.in_features,out_features=10,bias=True)
net = net.train().to(device=DEVICE, dtype=DTYPE)
print(net)

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop2): Dropout(p=0.0, inplace=F

In [11]:
optimizer = torch.optim.SGD(net.head.parameters(), lr=lr, momentum=momentum)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)
loss_func = torch.nn.CrossEntropyLoss()

In [12]:
def tran(epoch):
    net.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    scheduler.step()
    print('[%d] loss: %.3f' % (epoch + 1, running_loss))

In [13]:
def test():
    net.eval()
    all_counter=0
    correct_counter=0
    for i, data in enumerate(testloader, 0):
        inputs, labels = data
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        out = net(inputs)
        out = out.detach().cpu().argmax(1)
        t = labels.cpu()
        for m in range(len(t)):
            all_counter += 1
            if t[m] == out[m]:
                correct_counter += 1

    print(correct_counter, all_counter, correct_counter / all_counter)
    return (correct_counter / all_counter)

In [14]:
correctRate = 0
for i in range(5):
    tran(i)
    r = test()
    if(r > correctRate):
        correctRate = r
        print("best: ", r , " in NO: ", i)
        torch.save(net.cpu(),"checkpoint/trans_vit2.pth")
        net = net.to(DEVICE)

[1] loss: 107.572
9639 10000 0.9639
best:  0.9639  in NO:  0
[2] loss: 58.538
9641 10000 0.9641
best:  0.9641  in NO:  1
[3] loss: 51.534
9697 10000 0.9697
best:  0.9697  in NO:  2
[4] loss: 46.229
9686 10000 0.9686
[5] loss: 43.088
9695 10000 0.9695


In [15]:
optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=momentum)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.9)
loss_func = torch.nn.CrossEntropyLoss()

In [16]:
def tran(epoch):
    net.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    scheduler.step()
    print('[%d] loss: %.3f' % (epoch + 1, running_loss))

In [17]:
def test():
    net.eval()
    all_counter=0
    correct_counter=0
    for i, data in enumerate(testloader, 0):
        inputs, labels = data
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        out = net(inputs)
        out = out.detach().cpu().argmax(1)
        t = labels.cpu()
        for m in range(len(t)):
            all_counter += 1
            if t[m] == out[m]:
                correct_counter += 1

    print(correct_counter, all_counter, correct_counter / all_counter)
    return (correct_counter / all_counter)

In [ ]:
correctRate = 0
for i in range(epochs):
    tran(i)
    r = test()
    if(r > correctRate):
        correctRate = r
        print("best: ", r , " in NO: ", i)
        torch.save(net.cpu(),"checkpoint/trans_vit2.pth")
        net = net.to(DEVICE)

[1] loss: 60.277
9785 10000 0.9785
best:  0.9785  in NO:  0
[2] loss: 17.561
9819 10000 0.9819
best:  0.9819  in NO:  1
[3] loss: 5.403
9873 10000 0.9873
best:  0.9873  in NO:  2
[4] loss: 2.335
9863 10000 0.9863
[5] loss: 1.329
9873 10000 0.9873
[6] loss: 0.920
9875 10000 0.9875
best:  0.9875  in NO:  5
[7] loss: 0.568
9870 10000 0.987
[8] loss: 0.111
9881 10000 0.9881
best:  0.9881  in NO:  7
[9] loss: 0.044
9884 10000 0.9884
best:  0.9884  in NO:  8
[10] loss: 0.035
9886 10000 0.9886
best:  0.9886  in NO:  9
